In [1]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [2]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

In [10]:
#EKF NAV
idx_timeStamp = 0
idx_velNorth = 1
idx_velEast = 2
idx_velDown = 3
idx_velNorth_StdDev = 4
idx_velEast_StdDev = 5
idx_velDown_StdDev = 6
idx_Lat = 7
idx_Long = 8
idx_Alt = 9
idx_undulation = 10
idx_Lat_StdDev = 11
idx_Long_StdDev = 12
idx_Alt_StdDev = 13
idx_nav_status = 14

track1_Lat_0  = 48.06069 
track1_Long_0 = -1.99354
track1_Lat_1  = 48.05507 
track1_Long_1 = -2.02359

track2_Lat_0  = 48.06249 
track2_Long_0 = -1.99467
track2_Lat_1  = 48.05687 
track2_Long_1 = -2.02434

track3_Lat_0  = 48.06555 
track3_Long_0 = -1.99619
track3_Lat_1  = 48.06007 
track3_Long_1 = -2.02550

runaway = np.array([ [ 48.057546, -2.010483, 0.0 ],
          [ 48.058403, -2.005964, 0.0 ],
          [ 48.058191, -2.005869, 0.0 ],
          [ 48.057327, -2.010383, 0.0 ],
          [ 48.057546, -2.010483, 0.0 ]
          ])

## Load navigation data SbgLogEkfNav

In [4]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_21-22_SBG/dataLogger"

logEkfNav  = prefix + "/session_0003/2017_09_22/13h00/" + "sbgLogEkfNav.dat"
logUtcData = prefix + "/session_0003/2017_09_22/13h00/" + "sbgLogUtcData.dat"

Load logEkfNav data 13h00

In [5]:
# logEkfNav
timeStamps = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_timeStamp )
Long = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Long )
Lat = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Lat )
Alt = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Alt )
vN, vE, vD = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [6]:
Vel = (vN**2 + vE**2 + vD**2)**0.5

### Plot navigation data jointly with track references

In [7]:
def plotRunaway( ax ):
    ax.plot(runaway[:,1], runaway[:,0], "og", markeredgecolor = 'black')

In [11]:
def plotLongLatAndTrackReferences( ax ):
    ax.plot( Long, Lat, 'gray' )
    ax.plot( [track1_Long_0, track1_Long_1], [track1_Lat_0, track1_Lat_1], "o-.b", markeredgecolor = 'black' )
    ax.plot( [track2_Long_0, track2_Long_1], [track2_Lat_0, track2_Lat_1], "o-.b", markeredgecolor = 'black' )
    ax.plot( [track3_Long_0, track3_Long_1], [track3_Lat_0, track3_Lat_1], "o-.b", markeredgecolor = 'black' )

In [12]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

plt.title("Navigation data and track references 2017_09_22 13h00")

### Check the existence of record files

In [13]:
firstRecord = 960
lastRecord = 3372
aux = np.zeros(3372)
fileList = []
dataDir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_22_15_21_51"
for k in range(firstRecord, lastRecord, blocksPerFile):
    filename = dataDir + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)
idxFileList = np.arange( len( fileList ) ) * blocksPerFile + firstRecord

In [14]:
plt.plot(idxFileList, fileList, 'o-')

In [15]:
# times recorded by Windows XP
t_0    = (15, 21, 51)

record0Start = 960
record0Stop = 1866

record1Start = 1944
record1Stop = 2082

record2Start = 2190
record2Stop = 3372

Look for the time data corresponding to the records

In [16]:
hourOffset = 2

#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8
utc_h, utc_m, utc_s, utc_nano = np.loadtxt( logUtcData, skiprows = 1, 
                                           usecols = (idx_h, idx_m, idx_s, idx_nano), unpack=True )
utc_seconds = (utc_h + hourOffset) * 3600 + utc_m * 60 + utc_s + utc_nano * 1e-9

In [17]:
def getTimeOfDay( seconds ):
    h = np.floor( seconds / 3600 )
    m = np.floor( (seconds - h *3600) / 60 )
    s = seconds - h * 3600 - m * 60
    return (h, m, s)

def getSeconds( timeOfDay ):
    return (timeOfDay[0] * 3600 + timeOfDay[1] * 60 + timeOfDay[2])

def getIndices( start, stop ):
    idx_start = np.amax( np.where( utc_seconds <= getSeconds(start) ) )
    idx_stop = np.amax( np.where( utc_seconds <= getSeconds(stop) ) )
    return (idx_start, idx_stop)

In [18]:
# compute time of day using the date of block 0 and a computed delta

record0_tStart = getTimeOfDay( getSeconds( t_0 ) + record0Start / blocksPerFile * T_files )
record0_tStop = getTimeOfDay( getSeconds( t_0 ) + record0Stop / blocksPerFile * T_files )

record1_tStart = getTimeOfDay( getSeconds( t_0 ) + record1Start / blocksPerFile * T_files )
record1_tStop = getTimeOfDay( getSeconds( t_0 ) + record1Stop / blocksPerFile * T_files )

record2_tStart = getTimeOfDay( getSeconds( t_0 ) + record2Start / blocksPerFile * T_files )
record2_tStop = getTimeOfDay( getSeconds( t_0 ) + record2Stop / blocksPerFile * T_files )

In [20]:
record0_idx = getIndices (record0_tStart, record0_tStop)
record1_idx = getIndices (record1_tStart, record1_tStop)
record2_idx = getIndices (record2_tStart, record2_tStop)

### Plot navigation data and record periods

In [21]:
def addOnPlot(ax, idx, color, label=''):
    idxStart = idx[0]
    idxStop = idx[1]
    ax.plot( Long[idxStart:idxStop], Lat[idxStart:idxStop], '--', color=color, label=label )
    ax.plot( Long[idxStart], Lat[idxStart], 'D', color=color, markeredgecolor = 'black' )
    ax.plot( Long[idxStop], Lat[idxStop], 'D', color=color, markeredgecolor = 'black'  )

In [22]:
def addSpot(ax, idx, color):
    ax.plot( Long[idx], Lat[idx], 'D', color=color, markeredgecolor = 'black' )

In [25]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )
plotRunaway( ax )

addOnPlot( ax, record0_idx, 'yellow', 'record 0' )
addOnPlot( ax, record1_idx, 'red', 'record 1' )
addOnPlot( ax, record2_idx, 'magenta', 'record 2' )

plt.title("Navigation data, track references, record locations\nFlight 2017_09_22 13h00")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

## Track 1

In [26]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long, '.')
ax1.plot(np.ones(Long.shape) * track1_Long_0)
ax1.plot(np.ones(Long.shape) * track1_Long_1)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat, '.')
ax2.plot(np.ones(Lat.shape) * track1_Lat_0)
ax2.plot(np.ones(Lat.shape) * track1_Lat_1)

### Track 1 reference / Flight over 1

In [ ]:
idx_allTr3_fo1 = (515398, 527835)

In [31]:
# 960
start = 960
stop = start + 6 * 15
spot1_0 = getTimeOfDay( getSeconds( t_0 ) + start / blocksPerFile * T_files )
spot1_1 = getTimeOfDay( getSeconds( t_0 ) + stop / blocksPerFile * T_files )
idx_spot1_0 = np.amax( np.where( utc_seconds <= getSeconds(spot1_0) ) )
idx_spot1_1 = np.amax( np.where( utc_seconds <= getSeconds(spot1_1) ) )
idx_spot1 = (idx_spot1_0, idx_spot1_1)
velocity_spot1_0_spot1_1 = np.average( Vel[idx_spot1_0:idx_spot1_1] )
altitude_spot1_0_spot1_1 = np.average( Alt[idx_spot1_0:idx_spot1_1] )

In [34]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )
plotRunaway( ax )

addOnPlot( ax, idx_spot1, 'red', 'selection' )
addSpot( ax, idx_spot1_0, 'white' )
addSpot( ax, idx_spot1_1, 'white' )

plt.title("Navigation data and track references 2017_09_22 13h00\nTrack 1 Flight over 1")
plt.legend()
plt.grid()

# Data selection

In [ ]:
def getBlockNumberFromUtcSeconds( seconds, t_0 ):
    delta = seconds - getSeconds(t_0)
    return int( np.floor( delta / T_files ) * blocksPerFile )

In [ ]:
def getAltitudeVelocity( Alt, Vel, idx0, idx1 ):
    altitude = np.average( Alt[ idx0 : idx1 ] )
    velocity = np.average( Vel[ idx0 : idx1 ] )
    print("(altitude, velocity) = ({:.3f}, {:.3f})".format(altitude, velocity) )

In [ ]:
def getSelection(utc_seconds, Alt, Vel, idx):
    (idx0, idx1) = idx
    
    selection0 = getBlockNumberFromUtcSeconds( utc_seconds[idx0], t_0 )
    print( "Data selection starts at file: " + '{}'.format(selection0) )
    
    selection1 = getBlockNumberFromUtcSeconds( utc_seconds[idx1], t_0 )
    nb = int( np.floor( (selection1 - selection0) / blocksPerFile ) )
    print( "Data selection stops at file: " + '{}'.format(selection1) 
          + ", number of files in the selection: " + '{}'.format(nb+1) )
    
    getAltitudeVelocity( Alt, Vel, idx0, idx1 )

## Track 1

### Runaway Flight over 1

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_tr1_fo1 )

### Runaway Flight over 2

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_tr1_fo2 )

### Track 1 Flight over 1

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_allTr1_fo1 )

### Track 1 Flight over 2

In [ ]:
getSelection(utc_seconds, Alt, Vel, idx_allTr1_fo2 )

### Track 3 Flight over 1

In [ ]:
getSelection(utc_seconds, Alt, Vel, (idx_spot3_0, idx_spot3_1) )

# Annexe

### Check the dates of the recorded files

In [ ]:
def getLastLine( filename ):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            last = next(reader)
        print(last)

In [ ]:
def printUtc( index ):
    print( str(utc_h[index]) 
          + " : " + str(utc_m[index]) 
          + " : " + "{:.3f}".format(utc_s[index] + utc_nano[index] * 1e-9) )

In [ ]:
def getDataAtIndex(filename, index):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for r in range(index+1):
            next(reader)
        val = next(reader)
    return val

In [ ]:
def getBlockNumber( t, t_0 ):
    delta = getSeconds( t ) - getSeconds(t_0)
    return int( np.floor( delta / T_files ) * blocksPerFile )

def getUtcIndex( tod ):
    return np.amax( np.where( utc_seconds <= getSeconds( tod ) ) )

def getUtcData( logUtcData, timeOfDay ):
    with open( logUtcData, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        utc = np.loadtxt( io.StringIO( next(reader)[0] ))
        
        while (utc[5] < timeOfDay[0] - hourOffset):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
        while (utc[6] < timeOfDay[1]):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
        while (utc[7] < timeOfDay[2]):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
    return utc